<a href="https://colab.research.google.com/github/pchinso/garminfit_generator/blob/main/Te_damos_la_bienvenida_a_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
from xml.dom import minidom

# Added arguments to the function definition
def create_workout(warmup_duration=600, interval_duration=240, recovery_duration=180, cooldown_duration=600, interval_count=4, work_hr_min=85, work_hr_max=95, recovery_hr_min=60, recovery_hr_max=70):
    # Create XML structure
    root = ET.Element('TrainingCenterDatabase', {
        'xmlns': 'http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2',
        'xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
        'xsi:schemaLocation': 'http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2 http://www.garmin.com/xmlschemas/TrainingCenterDatabasev2.xsd'
    })

    workouts = ET.SubElement(root, 'Workouts')
    workout = ET.SubElement(workouts, 'Workout', {'Name': 'VO2Max 4x4 Noruego'})

    # Metadata
    sport_type = ET.SubElement(workout, 'SportType')
    sport_type.text = 'Running'

    # 1. Warm-up
    warmup = ET.SubElement(workout, 'Step')
    warmup.set('xsi:type', 'Warmup_t')
    warmup_duration_elem = ET.SubElement(warmup, 'Duration')
    warmup_duration_elem.set('xsi:type', 'Time_t')
    # Use the warmup_duration argument
    ET.SubElement(warmup_duration_elem, 'Seconds').text = str(warmup_duration)
    ET.SubElement(warmup, 'Intensity').text = 'Active'

    # 2. Interval blocks
    # Use the interval_count argument
    for i in range(1, interval_count + 1):
        # Work interval
        work = ET.SubElement(workout, 'Step')
        work.set('xsi:type', 'Workout_t')
        work_duration_elem = ET.SubElement(work, 'Duration')
        work_duration_elem.set('xsi:type', 'Time_t')
        # Use the interval_duration argument
        ET.SubElement(work_duration_elem, 'Seconds').text = str(interval_duration)
        ET.SubElement(work, 'Intensity').text = 'Active'

        # HR target for work interval
        work_target = ET.SubElement(work, 'Target')
        work_target.set('xsi:type', 'HeartRate_t')
        work_zone = ET.SubElement(work_target, 'HeartRateZone')
        # Use the work_hr_min and work_hr_max arguments
        ET.SubElement(work_zone, 'Low').text = str(work_hr_min)
        ET.SubElement(work_zone, 'High').text = str(work_hr_max)

        # Only add recovery after first three intervals (or based on interval_count)
        if i < interval_count:
            # Recovery interval
            recovery = ET.SubElement(workout, 'Step')
            recovery.set('xsi:type', 'Workout_t')
            recovery_duration_elem = ET.SubElement(recovery, 'Duration')
            recovery_duration_elem.set('xsi:type', 'Time_t')
            # Use the recovery_duration argument
            ET.SubElement(recovery_duration_elem, 'Seconds').text = str(recovery_duration)
            ET.SubElement(recovery, 'Intensity').text = 'Active'

            # HR target for recovery
            recovery_target = ET.SubElement(recovery, 'Target')
            recovery_target.set('xsi:type', 'HeartRate_t')
            recovery_zone = ET.SubElement(recovery_target, 'HeartRateZone')
            # Use the recovery_hr_min and recovery_hr_max arguments
            ET.SubElement(recovery_zone, 'Low').text = str(recovery_hr_min)
            ET.SubElement(recovery_zone, 'High').text = str(recovery_hr_max)

    # 3. Cool-down
    cooldown = ET.SubElement(workout, 'Step')
    cooldown.set('xsi:type', 'Cooldown_t')
    cooldown_duration_elem = ET.SubElement(cooldown, 'Duration')
    cooldown_duration_elem.set('xsi:type', 'Time_t')
    # Use the cooldown_duration argument
    ET.SubElement(cooldown_duration_elem, 'Seconds').text = str(cooldown_duration)
    ET.SubElement(cooldown, 'Intensity').text = 'Active'

    # Generate XML
    xml_str = ET.tostring(root, encoding='utf-8')
    pretty_xml = minidom.parseString(xml_str).toprettyxml(indent='  ')
    return pretty_xml

# Generate and save the default workout (using default arguments)
tcx_content = create_workout()
with open("VO2Max_4x4_Noruego.tcx", "w", encoding='utf-8') as tcx_file:
    tcx_file.write(tcx_content)

print("Workout file generated: VO2Max_4x4_Noruego.tcx")

Workout file generated: VO2Max_4x4_Noruego.tcx


In [11]:
for week in range(1, 4):
    # Increase intensity each week
    work_hr_min = 85 + week
    work_hr_max = 95

    # Generate modified workout
    modified_content = create_workout(
        warmup_duration=600,
        interval_duration=240,
        recovery_duration=180,
        cooldown_duration=600,
        interval_count=4,
        work_hr_min=work_hr_min,
        work_hr_max=work_hr_max,
        recovery_hr_min=60,
        recovery_hr_max=70
    )

    # Save with week identifier
    filename = f"VO2Max_4x4_Noruego_Week{week}.tcx"
    with open(filename, "w") as f:
        f.write(modified_content)

In [15]:
def format_pace(min_per_km):
    """Convert min/km pace to mm:ss format for Zwift"""
    minutes = int(min_per_km)
    seconds = int((min_per_km - minutes) * 60)
    return f"{minutes}:{seconds:02d}"

def create_zwo_workout():
    # Calculate paces for different intensities
    warmup_pace_min = format_pace(5.5)   # 5:30 min/km
    warmup_pace_max = format_pace(6.5)   # 6:30 min/km
    interval_paces = [
        (format_pace(3.75), format_pace(4.25)),  # Interval 1: 3:45-4:15
        (format_pace(3.70), format_pace(4.10)),  # Interval 2: 3:42-4:06
        (format_pace(3.65), format_pace(4.05)),  # Interval 3: 3:39-4:03
        (format_pace(3.60), format_pace(4.00))   # Interval 4: 3:36-4:00
    ]
    recovery_pace_min = format_pace(6.0)  # 6:00 min/km
    recovery_pace_max = format_pace(6.5)  # 6:30 min/km

    zwo_template = f"""<workout_file>
    <author>VO2Max 4x4 Norwegian Generator</author>
    <name>VO2Max 4x4 Noruego</name>
    <description>Norwegian 4x4 VO2 Max Intervals - 4x(4min @ 85-95% HRmax + 3min recovery)</description>
    <sportType>run</sportType>
    <category>Intervalos</category>
    <tags>
        <tag name="Intensidad">Alta</tag>
        <tag name="Duración">Medio</tag>
    </tags>

    <workout>
        <!-- Calentamiento: 10 min trote suave -->
        <Warmup Duration="600" PowerLow="0.40" PowerHigh="0.55"
                 PaceLow="{warmup_pace_min}" PaceHigh="{warmup_pace_max}" cadence="75-85" />

        <!-- Bloques de intervalo (4x) -->
        <SteadyState Duration="240" Power="0.85"
                     PaceLow="{interval_paces[0][0]}" PaceHigh="{interval_paces[0][1]}"
                     cadence="85-95" HeartRate="85-95">
            <text>Intervalo 1 - 4 min @ VO2 Max</text>
        </SteadyState>
        <SteadyState Duration="180" Power="0.55"
                     PaceLow="{recovery_pace_min}" PaceHigh="{recovery_pace_max}"
                     cadence="75-85" HeartRate="60-70">
            <text>Recuperación 1 - 3 min</text>
        </SteadyState>

        <SteadyState Duration="240" Power="0.87"
                     PaceLow="{interval_paces[1][0]}" PaceHigh="{interval_paces[1][1]}"
                     cadence="85-95" HeartRate="85-95">
            <text>Intervalo 2 - 4 min @ VO2 Max</text>
        </SteadyState>
        <SteadyState Duration="180" Power="0.55"
                     PaceLow="{recovery_pace_min}" PaceHigh="{recovery_pace_max}"
                     cadence="75-85" HeartRate="60-70">
            <text>Recuperación 2 - 3 min</text>
        </SteadyState>

        <SteadyState Duration="240" Power="0.89"
                     PaceLow="{interval_paces[2][0]}" PaceHigh="{interval_paces[2][1]}"
                     cadence="85-95" HeartRate="85-95">
            <text>Intervalo 3 - 4 min @ VO2 Max</text>
        </SteadyState>
        <SteadyState Duration="180" Power="0.55"
                     PaceLow="{recovery_pace_min}" PaceHigh="{recovery_pace_max}"
                     cadence="75-85" HeartRate="60-70">
            <text>Recuperación 3 - 3 min</text>
        </SteadyState>

        <SteadyState Duration="240" Power="0.91"
                     PaceLow="{interval_paces[3][0]}" PaceHigh="{interval_paces[3][1]}"
                     cadence="85-95" HeartRate="85-95">
            <text>Intervalo 4 - 4 min @ VO2 Max</text>
        </SteadyState>

        <!-- Enfriamiento: 10 min trote suave -->
        <Cooldown Duration="600" PowerLow="0.40" PowerHigh="0.55"
                  PaceLow="{warmup_pace_min}" PaceHigh="{warmup_pace_max}" cadence="75-85" />
    </workout>
</workout_file>"""
    return zwo_template

# Generate and save the workout
zwo_content = create_zwo_workout()
with open("VO2Max_4x4_Noruego.zwo", "w", encoding='utf-8') as zwo_file:
    zwo_file.write(zwo_content)

print("Workout file generated: VO2Max_4x4_Noruego.zwo")
print("Place in: Documents/Zwift/Workouts (Windows) or ~/Documents/Zwift/Workouts (Mac)")

Workout file generated: VO2Max_4x4_Noruego.zwo
Place in: Documents/Zwift/Workouts (Windows) or ~/Documents/Zwift/Workouts (Mac)
